In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llm.pl.tester.integer import test, int_dsl as dsl
from llm.learn.codegpt import CodeNet
from llm.learn.serialize import deserialize, extract_code

#codenet = CodeNet.from_dsl(dsl).to('cuda:0')
codenet = CodeNet.from_pretrained('../tmp_trainer').to('cuda:0')

In [ ]:
print(codenet.config)

In [ ]:
from llm.pl.tester.integer import incr, decr

codes = [test, incr, decr]
data = codenet.tokenizer.batchify(codes)

In [ ]:
tokenizer = codenet.tokenizer
for idx, i in enumerate(codes):
    A = str(extract_code(i))
    B = str(deserialize(tokenizer.decode(data['input_ids'][idx])))
    assert A == B

In [ ]:
codenet.lm.transformer.wpe = None
codenet.lm.transformer.wte = None
output = codenet(**data)
# print(output)

In [ ]:
#len(output['past_key_values'])
#print(output.logits.shape)
output.logits.argmax(axis=-1)

In [ ]:
from llm.learn.codegpt import SampleState

state = SampleState.new(1)
for i in range(34):
    state = codenet.next(state, tokens=None, sample_mode='BEST')
state = SampleState.new(1)
for i in range(40):
    tokens = ['car'] if i == 2 else None
    # tokens = None
    state = codenet.next(state, tokens=tokens, sample_mode='BEST')

assert str(state.codes[0]) == str(extract_code(test))

print(state.codes[0])
print(state.codes[0].serialize())